In [1]:
from pathlib import Path
from tqdm.notebook import tqdm
from tabulate import tabulate
from omnibelt import human_readable_number
import omnifig as fig
from sink import database, misc
import humanize
import pandas as pd
fig.initialize()

In [7]:
db = fig.create_config(_type='file-db', db_path=misc.data_root() / 'largetest.db').pull(silent=True)
db

In [8]:
itr = db.find_all()

In [9]:
rows = []
for raw in db.find_all():
	path, (hsh, (size, modtime)) = raw
	rows.append({
		'path': path,
		'hash': hsh,
		'size': size,
		'modtime': modtime,
	})
df = pd.DataFrame(rows)
df['isfile'] = df['path'].apply(lambda p: Path(p).is_file())
len(df)

6286

In [10]:
base_path = Path(min(df['path'], key=lambda p: len(Path(p).absolute().parts)))

In [13]:
empty = df[df['size'] == 0]['path']
len(empty)

123

In [11]:
dupes = {hsh: paths for hsh, paths in df.groupby('hash')['path'].apply(list).to_dict().items() if len(paths) > 1}
print(tabulate([[len(paths), '\n'.join(str(Path(p).relative_to(base_path)) for p in paths)] for hsh, paths in sorted(dupes.items(), key=lambda item: (item[1], item[0]))], headers=['num', 'paths']))

  num  paths
-----  -------------------------------------------------------------------------------------------------------------------------------------------------------
    2  Artemis\Documents\3D Pokemon
       Orion\Documents\3D Pokemon
    2  Artemis\Documents\3D Pokemon\3D Pokemon.pptx
       Orion\Documents\3D Pokemon\3D Pokemon.pptx
    2  Artemis\Documents\3D Pokemon\Pikachu 1.png
       Orion\Documents\3D Pokemon\Pikachu 1.png
    2  Artemis\Documents\3D Pokemon\Pikachu 2.png
       Orion\Documents\3D Pokemon\Pikachu 2.png
    2  Artemis\Documents\3D Pokemon\Pokeball.png
       Orion\Documents\3D Pokemon\Pokeball.png
    2  Artemis\Documents\Airsoft Guns.pptx
       Orion\Documents\Airsoft Guns.pptx
    2  Artemis\Documents\Board Games.pub
       Orion\Documents\Board Games.pub
    2  Artemis\Documents\Budget Planning Sheet Present.docx
       Orion\Documents\Budget Planning Sheet Present.docx
    2  Artemis\Documents\Building a Character.docx
       Orion\Documents\Building

In [26]:
current_base_size = (df[df['path'] == str(base_path)])['size'].item()
print(humanize.naturalsize(current_base_size, binary=True, format="%.3g"))

12.4 GiB


In [27]:
surplus_size = sum([df[df['path'] == paths[0]]['size'].item() * (len(paths)-1) for paths in dupes.values()])
print(humanize.naturalsize(surplus_size, binary=True, format="%.3g"))

965 MiB


In [23]:
# count number of files
num_files = len(df[df['isfile']])
num_dirs = len(df[~df['isfile']])
num_dirs, num_files

(386, 5900)